## Examples of usage

In [1]:
import numpy as np
import random
import pandas as pd
import meliora.meliora.core as vt

In [2]:
# Import data
test_data_main = pd.read_csv('pd_test_data_main.csv')
test_data_add = pd.read_csv('pd_test_data_period_2.csv')
test_pd_psi = pd.read_csv('pd_bellini.csv')
german_data = pd.read_csv('german_data.csv')

In [3]:
# Binomial test
vt.binomial_test(test_data_main, "ratings", "default_flag", "predicted_pd")

,Rating class,Predicted PD,Total count,Defaults,Actual Default Rate,p_value,Reject H0
0,A,0.103429,406,55,0.135468,0.023892,True
1,B,0.099524,480,41,0.085417,0.867441,False
2,C,0.095719,114,10,0.087719,0.660553,False


In [4]:
# Jeffrey's test
vt.jeffreys_test(test_data_main, 'ratings', 'default_flag', 'predicted_pd', 0.05)

,Rating class,Predicted PD,Total count,Defaults,Actual Default Rate,p_value,Reject H0
0,A,0.103429,406,55,0.135468,0.019959,True
1,B,0.099524,480,41,0.085417,0.849552,False
2,C,0.095719,114,10,0.087719,0.598649,False


In [5]:
# Brier score
vt.brier_score(test_data_main, "ratings", "default_flag", "predicted_pd")

0.0012895084997917349

In [6]:
# Hosmer-Lemeshow test
vt.hosmer_test(test_data_main, "ratings", "default_flag", "predicted_pd")

[0.1302500370047408, False]

In [7]:
# Spiegelhalter test
vt.spiegelhalter_test(test_data_main, "ratings", "default_flag", "predicted_pd")

(-0.6637590511485174, False)

In [8]:
# ROC test
vt.roc_auc(test_data_main, "default_flag", "predicted_pd")

0.5008547549702419

In [9]:
# Spearman correlation
# vt.spearman(test_data_main, "default_flag", "predicted_pd")

In [10]:
x = [1,2,3,4,5]
y = [5,6,7,8,7]
vt.spearman_corr(x, y).correlation

0.8207826816681233

In [11]:
# Somers D
x = [0, 1, 1, 1, 1] 
y = [1, 1, 1, 0, 1]
vt.somersd(x, y)

SomersDResult(statistic=-0.25, pvalue=0.3613104285261787, table=array([[0, 1],
       [1, 3]]))

In [12]:
# Kendall tau

In [13]:
x = [1, 2, 3, 2, 1, 3, 4, 2, 5, 2, 6, 5, 5]
y = [5, 5, 6, 2, 1, 4, 4, 2, 1, 2, 1, 5, 5]

tau, pvalue = vt.kendall_tau(x, y)
tau

0.030306509211290782

In [14]:
# PSI test
test_pd_psi['remaining_mat' + "_bin"] = pd.cut(test_pd_psi['remaining_mat'], bins=10, labels=False)
test_pd_psi['year_bins'] = np.where(test_pd_psi['vintage_year'] < 2007, 'period_1', 'period_2')

# Export data
test_pd_psi.to_csv('test_pd_psi.csv', index=False)

In [15]:
kala = vt.psi(test_pd_psi, 'year_bins', 'remaining_mat_bin')
kala[1]

1.0344129494141174

In [16]:
zen = vt.calc_iv(german_data, 'checkingstatus', 'GoodCredit')



In [17]:
zen[1]

0.6660115033513336

In [18]:
zen[0]

,Variable,Value,All,Bad,Share,Bad Rate,Distribution Good,Distribution Bad,WoE,IV
0,checkingstatus,A11,274,135,0.274,0.492701,0.198571,0.450000,-0.818099,0.205693
1,checkingstatus,A12,269,105,0.269,0.390335,0.234286,0.350000,-0.401392,0.046447
3,checkingstatus,A13,63,14,0.063,0.222222,0.070000,0.046667,0.405465,0.009461
2,checkingstatus,A14,394,46,0.394,0.116751,0.497143,0.153333,1.176263,0.404410


In [19]:
# Transform input data into the required format
df = test_data_main.groupby('ratings').agg({'predicted_pd': "mean", 'default_flag': ["count", "sum", "mean"]})
df.columns = ["PD", "N", "D", "Default Rate"]


In [20]:
realised_values = df["Default Rate"]
predicted_values = df["PD"]

In [21]:
from scipy.stats import norm

# Calculate mean squared error
errors = realised_values - predicted_values
mse = (errors**2).sum() / len(errors)

# # Calculate null expectation and variance of MSE
expectations = sum(predicted_values * (1 - predicted_values)) / len(realised_values)
variances = sum(predicted_values * (1 - 2 * predicted_values)**2 * (1 - predicted_values)) / len(realised_values)**2

# Calculate standardized statistic
z_score = (mse - expectations) / np.sqrt(variances)  # todo: check formula

# Calculate standardized MSE as test statistic, then its p-value
outcome = z_score > norm.ppf(1 - 0.05/2)

In [22]:
errors

ratings
A    0.032039
B   -0.014107
C   -0.008000
dtype: float64

In [23]:
mse

0.00042983616659724496

In [24]:
norm.ppf(1 - 0.05/2)

1.959963984540054

In [25]:
help(df.groupby)

Help on method groupby in module pandas.core.frame:

groupby(by=None, axis: 'Axis' = 0, level: 'Level | None' = None, as_index: 'bool' = True, sort: 'bool' = True, group_keys: 'bool' = True, squeeze: 'bool | lib.NoDefault' = <no_default>, observed: 'bool' = False, dropna: 'bool' = True) -> 'DataFrameGroupBy' method of pandas.core.frame.DataFrame instance
    Group DataFrame using a mapper or by a Series of columns.
    
    A groupby operation involves some combination of splitting the
    object, applying a function, and combining the results. This can be
    used to group large amounts of data and compute operations on these
    groups.
    
    Parameters
    ----------
    by : mapping, function, label, or list of labels
        Used to determine the groups for the groupby.
        If ``by`` is a function, it's called on each value of the object's
        index. If a dict or Series is passed, the Series or dict VALUES
        will be used to determine the groups (the Series' values

In [26]:
test_data_main.head()

,ratings,default_flag,predicted_pd
0,B,0,0.137215
1,A,0,0.130365
2,C,0,0.188544
3,A,0,0.178812
4,C,0,0.135670


In [27]:
# hosmer_lemeshow = function(y, prob, g, stat_type){
#   mtx = cbind(y, y_not = 1- y, prob, prob_not = 1-prob)
#   mtx = as.data.frame(mtx)
#   mtx = mtx[order(mtx$prob),]
#   n <- length(prob)/g
#   nr <- nrow(mtx)
  
#   ## C statistics, same number of instances in each bin
#   if (stat_type == 'C'){
#     split_mtx = split(mtx, rep(1:ceiling(nr/n), each=n, length.out=nr))
#   }else{ ### H statistics, equal intervals
#     split_mtx = split(mtx, cut(mtx$prob, seq(0,1,1/g), include.lowest=TRUE))
#     split_mtx = split_mtx[sapply(split_mtx, nrow)>0]
#     ###
#   }
  
#   H_stat = 0
#   for (i in 1:length(split_mtx)){
#     obs = sum(split_mtx[[i]]$y == 1)
#     exp = sum(split_mtx[[i]]$prob)
#     obs_not = sum(split_mtx[[i]]$y == 0)
#     exp_not = sum(split_mtx[[i]]$prob_not)
    
#     if (exp == 0 || exp_not == 0){
#       next
#     }
    
#     bin_sum = ((obs - exp)**2)/exp + ((obs_not - exp_not)**2)/exp_not

#     H_stat = H_stat + bin_sum
#   }
#   PVAL = 1 - pchisq(H_stat, g - 2)
  
#   cat('PVALUE', PVAL, '\n')
#   cat('stat', H_stat, '\n')
#   return(PVAL)
# }

In [28]:
# Spiegelhalter_z = function(y, prob){
#   alpha = 0.05
#   z_score = sum((y-prob)*(1-2*prob))/sqrt(sum(((1-2*prob)^2)*prob*(1-prob)))
#   print(z_score)
#   if (abs(z_score) > qnorm(1-alpha/2)){
#     print('reject null. NOT calibrated')
#   } else{
#     print('fail to reject. calibrated')
#   }
#   cat('z score: ', z_score, '\n')
#   cat('p value: ', 1-pnorm(abs(z_score)), '\n')
#   return(z_score)
# }

In [29]:
# https://documentation.sas.com/doc/en/pgmsascdc/v_011/statug/statug_code_logiex19.htm

In [30]:
# ##Entropy
# def entropy(Y):
#     """
#     Also known as Shanon Entropy
#     Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
#     """
#     unique, count = np.unique(Y, return_counts=True, axis=0)
#     prob = count/len(Y)
#     en = np.sum((-1)*prob*np.log2(prob))
#     return en


# #Joint Entropy
# def jEntropy(Y,X):
#     """
#     H(Y;X)
#     Reference: https://en.wikipedia.org/wiki/Joint_entropy
#     """
#     YX = np.c_[Y,X]
#     return entropy(YX)

# #Conditional Entropy
# def cEntropy(Y, X):
#     """
#     conditional entropy = Joint Entropy - Entropy of X
#     H(Y|X) = H(Y;X) - H(X)
#     Reference: https://en.wikipedia.org/wiki/Conditional_entropy
#     """
#     return jEntropy(Y, X) - entropy(X)


# #Information Gain
# def gain(Y, X):
#     """
#     Information Gain, I(Y;X) = H(Y) - H(Y|X)
#     Reference: https://en.wikipedia.org/wiki/Information_gain_in_decision_trees#Formal_definition
#     """
#     return entropy(Y) - cEntropy(Y,X)

In [31]:
# https://machinelearningmastery.com/divergence-between-probability-distributions/

In [32]:
# https://medium.com/@monadsblog/the-kullback-leibler-divergence-5071c707a4a6

In [33]:
df.

SyntaxError: invalid syntax (791285630.py, line 1)